# A Complete Example of Training a Neural Network

In this example we will go throughout usual steps when applying a neural network for predicting some target attribute based on known data.

## Problem Statement
The aim is to build a classification model for a bank that should predict whether an applicant should get a loan. We will be using the dataset which contains 614 observations and 13 variables, as described below:

1. Loan_ID - ID of the application - we will omit it.
2. Gender - gender of the applicant.
3. Married - marital status of the applicant.
4. Dependents - how many persons (children) depends on the income of the applicant.
5. Education - highest education passed be the applicant.
6. Self_Employed - whether the applicant is self-employed.
7. ApplicantIncome - income of the applicant.
8. CoapplicantIncome - income of the co-applicant.
9. LoanAmount - how much would like the applicant to loan.
10. Loan_Amount_Term - in how many month should the loan be returned.
11. Credit_History - whether the applicant has had a loan recently. 
12. Property_Area - are in m^2 where the house will be built.
13. Loan_Status - decision whether the loan was approved


As we can see some attributes are of different types: integers, floats, strings and categorical.


## Evaluation Metric
We will use accuracy, which represents the percentage of cases correctly classified.

## Steps

1. Loading the required libraries and modules.

2. Reading the data and performing basic data checks and transformation.

3. Creating arrays for the features and the target variable.

4. Creating the training and test datasets.

5. Building, predicting, and evaluating the neural network model.

## Step 1 - Loading the Required Libraries and Modules

In [310]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

# Step 2 - Reading the Data and Performing Basic Data Checks
The first line of code reads in the data as pandas dataframe, while the second line prints the shape - 614 observations of 13 variables. The third line prints the first 5 lines of the table. Why we read the data using `pandas`? As the data attributes are of different types, many of them are not numerical.

In [306]:
df = pd.read_csv('loans.csv') 
print(df.shape)
df.head(20)

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [309]:
df['Gender'].value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

Next we look at a summary statistics of the variables.

Looking at the summary for the 'diabetes' variable, we observe that the mean value is 0.35, which means that around 35 percent of the observations in the dataset have diabetes. Therefore, the baseline accuracy is 65 percent and our neural network model should definitely beat this baseline benchmark.

In [245]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,5403.459283,6109.041673,150.0,2877.5,3812.5,5795.00,81000.0
CoapplicantIncome,614.0,1621.245798,2926.248369,0.0,0.0,1188.5,2297.25,41667.0
LoanAmount,592.0,146.412162,85.587325,9.0,100.0,128.0,168.00,700.0
Loan_Amount_Term,600.0,342.000000,65.120410,12.0,360.0,360.0,360.00,480.0
Credit_History,564.0,0.842199,0.364878,0.0,1.0,1.0,1.00,1.0


The attributes `LoanAmount` and `Credit_History` have lower counts. This indicates that there are some missing values in the table. The rest of the attributes not present in the description are attributes with non-numerical values.

## Resolving the Missing Values

There are several methods for dealing with missing values:
* omitting all records containing at least one missing value,
* replacing the missing values by mean or median value,
* replacing the missing value by a constant, optionally adding a new attribute marking the added data.

The missing values for `LoanAmount` we replace by a median value.

In [246]:
print("In LoanAmount we have", df['LoanAmount'].isnull().sum(),"missing values")

In LoanAmount we have 22 missing values


In [247]:
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
print("In LoanAmount we have", df['LoanAmount'].isnull().sum(),"missing values")
df.head(10)

In LoanAmount we have 0 missing values


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


Similarly, 
1. fill the missing values for `Credit_History` with zeros
2. fill the missing values for `Loan_Amount_Term` with the median loan amount.

In [248]:
# fill the missing values for Credit_History with zeros
...

print("In Credit_History we have", df['Credit_History'].isnull().sum(),"missing values")
df.head(10)

In Credit_History we have 0 missing values


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [249]:
# fill the missing values for Loan_Amount_Term with the median loan term
...

print("In Loan_Amount_Term we have", df['Loan_Amount_Term'].isnull().sum(),"missing values")
df.head(10)

In Loan_Amount_Term we have 0 missing values


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


## Converting Categorical Values into Integers

Neural networks work with numerical values only. Therefore, we must convert all categorical attributes into numbers. There are two kinds of categorical values
1. Ordered categories, when there is an ordering among the categories, like days in a week. Then the conversion should preserve the ordering. In this case, we can simply assign integers 0, 1, 2, ... for all categories in such a way that the integers preserve the same ordering as the original categories. 
2. Unordered categories, when there is no default ordering among the categories, like colors. In this case one-hot-encoding is used.




In [250]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


### Converting `Gender` attribute 

In [251]:
from pandas.api.types import CategoricalDtype

In [252]:
df['Gender'].unique()

array(['Male', 'Female', nan], dtype=object)

In [313]:
# fill the missing values for Gender with the mode - 'Male'
...

gender_categories = CategoricalDtype(['Female','Male'])
df['GenderInt'] = df['Gender'].astype(gender_categories).cat.codes
print(df['GenderInt'].unique())
df.tail()

[ 1  0 -1]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,GenderInt
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,0
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,1
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,1
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,1
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N,0


### Converting `Married` Attribute

We will replace missing values for `Married` with `No` and then we convert the attribute into integer.

In [254]:
# replace missing values for Married with No
...

print(df['Married'].unique())
print("In Married we have", df['Married'].isnull().sum(),"missing values")
# convert Married into MarriedInt by creating CategoricalDtype(['No', 'Yes'])
...
...
print(df['MarriedInt'].unique())
df.head()

['No' 'Yes']
In Married we have 0 missing values
[0 1]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,GenderInt,MarriedInt
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,1,0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,1,0


### Converting `Dependents` Attribute

In [255]:
print(df['Dependents'].unique())

['0' '1' '2' '3+' nan]


In [256]:
# replace missing values of Dependents by '3+'
...
print(df['Dependents'].unique())
print("In Dependents we have", df['Dependents'].isnull().sum(),"missing values")
dependents_categories = CategoricalDtype(['0', '1', '2', '3+'], ordered=True)
df['DependentsInt'] = df['Dependents'].astype(dependents_categories).cat.codes
print(df['DependentsInt'].unique())
df.head()

['0' '1' '2' '3+']
In Dependents we have 0 missing values
[0 1 2 3]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,GenderInt,MarriedInt,DependentsInt
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,1,0,0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,1,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,1,0,0


### Converting `Education` Attribute

In [257]:
print(df['Education'].unique())

['Graduate' 'Not Graduate']


In [258]:
# create a new attribute EducationInt with integer representation for Education attribute
...
...
print(df['EducationInt'].unique())
df.head()

[1 0]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,GenderInt,MarriedInt,DependentsInt,EducationInt
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,1,0,0,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,1
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,1,0,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,1,0,0,1


### Converting `Self_Employed` attribute

In [259]:
df['Self_Employed'].unique()

array(['No', 'Yes', nan], dtype=object)

In [260]:
df['Self_Employed'].fillna(df['Self_Employed'].mode(), inplace=True)
print("In Self_Employed we have", df['Self_Employed'].isnull().sum(),"missing values")
self_employed_categories = CategoricalDtype(['No', 'Yes'])
df['Self_EmployedInt'] = df['Self_Employed'].astype(self_employed_categories).cat.codes
print(df['Self_EmployedInt'].unique())
df.head()

In Self_Employed we have 32 missing values
[ 0  1 -1]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,GenderInt,MarriedInt,DependentsInt,EducationInt,Self_EmployedInt
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,1,0,0,1,0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,1,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,1,0,0,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,1,0,0,1,0


Filling the missing values failed.

In [261]:
df['Self_Employed'].mode()

0    No
dtype: object

`.mode()` returned a Series object, as in general `.mode()` can return more than one value. We will use the first value returned by `.mode()`.

In [262]:
print(df['Self_Employed'].unique())
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
print(df['Self_Employed'].unique())


['No' 'Yes' nan]
['No' 'Yes']


### Converting `Property_Area` attribute

In [263]:
df['Property_Area'].unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

The possible values of Property_Area attribute seem to be ordered - 'Urban', 'Semiurban' and 'Rural'. Convert them into inters 0, 1 and 2, respectively.

In [264]:
...
...
print(df['Property_AreaInt'].unique())
df.head()

[0 2 1]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,GenderInt,MarriedInt,DependentsInt,EducationInt,Self_EmployedInt,Property_AreaInt
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,1,0,0,1,0,0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,1,0,2
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,1,0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,1,0,0,0,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,1,0,0,1,0,0


### Converting `Loan_Status` attribute

In [265]:
df['Loan_Status'].unique()

array(['Y', 'N'], dtype=object)

In [266]:
loan_status_categories = CategoricalDtype(['N', 'Y'])
df['Loan_StatusInt'] = df['Loan_Status'].astype(loan_status_categories).cat.codes
print(df['Loan_StatusInt'].unique())
df.head()

[1 0]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,GenderInt,MarriedInt,DependentsInt,EducationInt,Self_EmployedInt,Property_AreaInt,Loan_StatusInt
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,1,0,0,1,0,0,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,1,1,1,1,0,2,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,1,0,1,1,0,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,1,1,0,0,0,0,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,1,0,0,1,0,0,1


### Final Check and Dropping Non-Numerical Attributes

In [267]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             614 non-null    object 
 2   Married            614 non-null    object 
 3   Dependents         614 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      614 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
 13  GenderInt          614 non-null    int8   
 14  MarriedInt         614 non-null    int8   
 15  DependentsInt      614 non-null    int8   
 16  EducationInt       614 non

In [268]:
df1 = df.drop(columns=['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
               'Property_Area','Loan_Status'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ApplicantIncome    614 non-null    int64  
 1   CoapplicantIncome  614 non-null    float64
 2   LoanAmount         614 non-null    float64
 3   Loan_Amount_Term   614 non-null    float64
 4   Credit_History     614 non-null    float64
 5   GenderInt          614 non-null    int8   
 6   MarriedInt         614 non-null    int8   
 7   DependentsInt      614 non-null    int8   
 8   EducationInt       614 non-null    int8   
 9   Self_EmployedInt   614 non-null    int8   
 10  Property_AreaInt   614 non-null    int8   
 11  Loan_StatusInt     614 non-null    int8   
dtypes: float64(4), int64(1), int8(7)
memory usage: 28.3 KB


## Step 3 - Creating Arrays for the Features and the Target Variable
The first line of code creates an object of the target variable called `target_column`. The second line gives us the list of all the features, excluding the target variable `Loan_StatusInt`.
The fourth line displays the summary of the data.

In [270]:
target_column = ['Loan_StatusInt'] 
predictors = list(set(list(df1.columns)) - set(target_column))
df1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,5403.459283,6109.041673,150.0,2877.50,3812.5,5795.00,81000.0
CoapplicantIncome,614.0,1621.245798,2926.248369,0.0,0.00,1188.5,2297.25,41667.0
LoanAmount,614.0,145.752443,84.107233,9.0,100.25,128.0,164.75,700.0
Loan_Amount_Term,614.0,342.410423,64.428629,12.0,360.00,360.0,360.00,480.0
Credit_History,614.0,0.773616,0.418832,0.0,1.00,1.0,1.00,1.0
GenderInt,614.0,0.817590,0.386497,0.0,1.00,1.0,1.00,1.0
MarriedInt,614.0,0.648208,0.477919,0.0,0.00,1.0,1.00,1.0
DependentsInt,614.0,0.817590,1.060618,0.0,0.00,0.0,2.00,3.0
EducationInt,614.0,0.781759,0.413389,0.0,1.00,1.0,1.00,1.0
Self_EmployedInt,614.0,0.081433,0.423472,-1.0,0.00,0.0,0.00,1.0


## Normalization of Values

The ranges of values of the first four attributes are much bigger that the ranges of the rest attributes. We will normalize all attributes. 

In [271]:
df1[predictors] = df1[predictors]/df1[predictors].max()
df1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,0.066709,0.075420,0.001852,0.035525,0.047068,0.071543,1.0
CoapplicantIncome,614.0,0.038910,0.070229,0.000000,0.000000,0.028524,0.055134,1.0
LoanAmount,614.0,0.208218,0.120153,0.012857,0.143214,0.182857,0.235357,1.0
Loan_Amount_Term,614.0,0.713355,0.134226,0.025000,0.750000,0.750000,0.750000,1.0
Credit_History,614.0,0.773616,0.418832,0.000000,1.000000,1.000000,1.000000,1.0
GenderInt,614.0,0.817590,0.386497,0.000000,1.000000,1.000000,1.000000,1.0
MarriedInt,614.0,0.648208,0.477919,0.000000,0.000000,1.000000,1.000000,1.0
DependentsInt,614.0,0.272530,0.353539,0.000000,0.000000,0.000000,0.666667,1.0
EducationInt,614.0,0.781759,0.413389,0.000000,1.000000,1.000000,1.000000,1.0
Self_EmployedInt,614.0,0.081433,0.423472,-1.000000,0.000000,0.000000,0.000000,1.0


`Self_EmployedInt` contains some values `-1`. We will leave them for now.

## Step 4 - Creating the Training and Test Datasets
The first couple of lines of code below create arrays of the independent (`X`) and dependent (`y`) variables, respectively. The third line splits the data into training and test dataset, and the fourth line prints the shape of the training and the test data.

In [272]:
X = df1[predictors].values
y = df1[target_column].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_test.shape",X_test.shape)
print("y_test.shape",y_test.shape)

X_train.shape (491, 11)
y_train.shape (491,)
X_test.shape (123, 11)
y_test.shape (123,)


## Step 5 - Building, Predicting, and Evaluating the Neural Network Model
In this step, we will build the neural network model using the scikit-learn library's estimator object, 'Multi-Layer Perceptron Classifier'. The first line of code (shown below) imports `MLPClassifier`.

The second line instantiates the model with the `hidden_layer_sizes` argument set to three layers, which has the same number of neurons as the count of features in the dataset. We will also select `relu` as the activation function and `adam` as the solver for weight optimization. 

The third line of code fits the model to the training data, while the fourth and fifth lines use the trained model to generate predictions on the training and test dataset, respectively.

In [273]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(random_state=42)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

Once the predictions are generated, we can evaluate the performance of the model. Being a classification algorithm, we will first import the required modules, which is done in the first line of code below. The second and third lines of code print the confusion matrix and the confusion report results on the training data.

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

The above output shows the performance of the model on training data. The accuracy is around 0.44 - this is the ratio of correct answers. Hence, the default values for the parameters of the network do not work well. Before tuning the model, let us check its performance on the test data.

In [ ]:
print(confusion_matrix(y_test,predict_test))

We can display the confusion matrix in a better form using `seaborn` module. If you do not have `seaborn` intsalled, you can do that, e.g., using

    pip install --user seaborn
    

In [ ]:
import seaborn as sn

cf_test_matrix = confusion_matrix(y_test, predict_test)
plt.figure(figsize=(5,3))
sn.heatmap(cf_test_matrix, annot=True, fmt='d')
plt.title('Confusion matrix for test data')
plt.xlabel('Predicted')
plt.ylabel('Actual')

In [ ]:
print(classification_report(y_test,predict_test))

Interestingly, the performance of the network on test data is better than on the train data.

# Fine-Tuning the Network

Try to optimize the network
* Is it possible to achieve higher accuracy on the test set?
* What is the simplest (smallest) network achieving your best accuracy on the test set?

You can use grid search combined with k-fold cross-validation.